In [2]:
import pandas as pd
import numpy as np
import random
import time

# Metody doboru sąsiedztwa

In [3]:
def generate_neighbor(solution, neighbor_type='swap'):
    i, j = np.random.choice(len(solution), size=2, replace=False)
    neighbor = solution.copy()
    city1, city2 = neighbor[i], neighbor[j]
    city_pair = [city1, city2]
    if neighbor_type == 'swap':
        # Generuj sąsiada przez zamianę dwóch losowych miast
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    elif neighbor_type == 'insert':
        # Generuj sąsiada przez wstawienie losowego miasta w inne miejsce
        neighbor.insert(j, neighbor.pop(i))
    elif neighbor_type == 'reverse':
        # Generuj sąsiada przez odwrócenie kolejności miast pomiędzy dwoma losowymi indeksami
        if i > j:
            i, j = j, i
        neighbor[i:j + 1] = reversed(neighbor[i:j + 1])
    else:
        raise ValueError("Nieznany rodzaj sąsiedztwa: {}".format(neighbor_type))
    return neighbor, city_pair

In [160]:
# Test funkcji generate_neighbor
solution = [1, 2, 3, 4, 5]
print("Rozwiązanie początkowe: {}".format(solution))
print("swap: {}".format(generate_neighbor(solution, neighbor_type='swap')))
print("insert: {}".format(generate_neighbor(solution, neighbor_type='insert')))
print("reverse: {}".format(generate_neighbor(solution, neighbor_type='reverse')))


Rozwiązanie początkowe: [1, 2, 3, 4, 5]
swap: ([1, 5, 3, 4, 2], [2, 5])
insert: ([1, 2, 3, 5, 4], [4, 5])
reverse: ([1, 2, 3, 5, 4], [5, 4])


# Funkcje trasy

In [4]:
# Stworzenie funkcji ustalenia początkowej trasy
def create_initial_route(distance_matrix):
    route = list(range(len(distance_matrix)))
    random.shuffle(route)
    return route

In [5]:
def calculate_total_distance(solution, distance_matrix):
    total_distance = 0
    num_cities = len(solution)

    for i in range(num_cities - 1):
        total_distance += distance_matrix.at[solution[i], solution[i + 1]]

    # Dodaj odległość z ostatniego do pierwszego miasta
    total_distance += distance_matrix.at[solution[num_cities - 1], solution[0]]

    return total_distance

# Tabu Search

In [6]:
def tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10):
    # Inicjalizacja bieżącego rozwiązania, najlepszego rozwiązania, listy tabu i najlepszego dystansu
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    tabu_list = []
    best_distance = calculate_total_distance(best_solution, distance_matrix)

    # Pętla główna algorytmu Tabu Search
    for _ in range(max_iterations):
        # Generowanie sąsiadów bieżącego rozwiązania
        neighbors = [generate_neighbor(current_solution, neighbor_type) for _ in range(gen_neighbor_size)]

        # Obliczanie dystansów dla kandydatów, którzy nie są na liście tabu
        candidate_distances = [(neighbor, city_pair, calculate_total_distance(neighbor, distance_matrix)) for neighbor, city_pair in neighbors if city_pair not in tabu_list]

        # Jeśli brak kandydatów spoza listy tabu, kontynuuj pętlę
        if not candidate_distances:
            continue

        # Wybieranie najlepszego kandydata
        candidate_solution, city_pair, candidate_distance = min(candidate_distances, key=lambda x: x[2])

        # Aktualizacja najlepszego rozwiązania, jeśli znaleziono lepsze
        if candidate_distance < best_distance:
            best_solution = candidate_solution
            best_distance = candidate_distance

        # Dodanie ruchu do listy tabu
        tabu_list.append(city_pair)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Usuwanie najstarszego ruchu z listy tabu

    # Zwracanie najlepszego rozwiązania i odpowiadającego mu dystansu
    return best_solution, best_distance


In [7]:
def tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10):
    # Inicjalizacja bieżącego rozwiązania, najlepszego rozwiązania, listy tabu i najlepszego dystansu
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    tabu_list = []
    best_distance = calculate_total_distance(best_solution, distance_matrix)

    # Początkowy czas wykonania algorytmu
    start_time = time.time()

    # Pętla główna algorytmu Tabu Search z ograniczeniem czasowym
    while (time.time() - start_time) < max_time_seconds:
        # Generowanie sąsiadów bieżącego rozwiązania
        neighbors = [generate_neighbor(current_solution, neighbor_type) for _ in range(gen_neighbor_size)]

        # Obliczanie dystansów dla kandydatów, którzy nie są na liście tabu
        candidate_distances = [(neighbor, city_pair, calculate_total_distance(neighbor, distance_matrix)) for neighbor, city_pair in neighbors if city_pair not in tabu_list]

        # Jeśli brak kandydatów spoza listy tabu, kontynuuj pętlę
        if not candidate_distances:
            continue

        # Wybieranie najlepszego kandydata
        candidate_solution, city_pair, candidate_distance = min(candidate_distances, key=lambda x: x[2])

        # Aktualizacja najlepszego rozwiązania, jeśli znaleziono lepsze
        if candidate_distance < best_distance:
            best_solution = candidate_solution
            best_distance = candidate_distance

        # Dodanie ruchu do listy tabu
        tabu_list.append(city_pair)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Usuwanie najstarszego ruchu z listy tabu

    # Zwracanie najlepszego rozwiązania i odpowiadającego mu dystansu
    return best_solution, best_distance


In [8]:
def tabu_search_print(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10):
    best_solution, best_distance = tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size)
    print("Tabu list size: {}, Max iterations: {}, Neighbor type: {}, Number of generated neighbors: {}".format(tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie: {}".format(best_solution))
    print("Długość trasy: {}".format(best_distance))

In [166]:
def tabu_search_with_time_limit_print(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10):
    best_solution, best_distance = tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size)
    print("Tabu list size: {}, Max time: {}, Neighbor type: {}, Number of generated neighbors: {}".format(tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie: {}".format(best_solution))
    print("Długość trasy: {}".format(best_distance))

In [9]:
def tabu_search_multiple_runs(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)
    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search i zapisz wyniki
        current_solution, current_distance = tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalna powtórzeń: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))

In [10]:
def tabu_search_with_time_limit_multiple_runs(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)

    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search_with_time_limit i zapisz wyniki
        current_solution, current_distance = tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalny czas liczenia: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))


## TSP_48

In [12]:
TSP_48 = pd.read_excel('Dane_TSP_48.xlsx', header=None)

### Sąsiedztwo

In [170]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000, 'swap', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [25, 11, 31, 38, 1, 35, 13, 6, 42, 32, 4, 14, 5, 17, 21, 23, 8, 10, 26, 2, 37, 24, 16, 12, 36, 18, 20, 41, 43, 7, 29, 19, 9, 28, 22, 46, 44, 33, 47, 15, 45, 27, 40, 0, 30, 39, 3, 34]
Długość trasy: 45768


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [25, 11, 31, 38, 1, 35, 13, 6, 42, 32, 4, 14, 5, 17, 21, 23, 8, 10, 26, 2, 37, 24, 16, 12, 36, 18, 20, 41, 43, 7, 29, 19, 9, 28, 22, 46, 44, 33, 47, 15, 45, 27, 40, 0, 30, 39, 3, 34]
Długość trasy: 45768


In [174]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [24, 12, 30, 22, 28, 4, 18, 40, 9, 14, 1, 39, 36, 5, 35, 42, 43, 7, 23, 13, 26, 17, 27, 19, 31, 3, 47, 46, 29, 38, 15, 16, 33, 44, 45, 6, 32, 8, 37, 10, 20, 34, 0, 21, 25, 41, 11, 2]
Długość trasy: 44636


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [24, 12, 30, 22, 28, 4, 18, 40, 9, 14, 1, 39, 36, 5, 35, 42, 43, 7, 23, 13, 26, 17, 27, 19, 31, 3, 47, 46, 29, 38, 15, 16, 33, 44, 45, 6, 32, 8, 37, 10, 20, 34, 0, 21, 25, 41, 11, 2]
Długość trasy: 44636

In [172]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000, 'reverse', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [39, 38, 17, 18, 46, 0, 43, 11, 7, 37, 28, 31, 9, 20, 10, 41, 6, 14, 13, 22, 23, 15, 24, 19, 27, 44, 34, 3, 42, 2, 35, 29, 30, 40, 36, 21, 26, 4, 45, 5, 32, 1, 16, 33, 47, 25, 8, 12]
Długość trasy: 47942


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [39, 38, 17, 18, 46, 0, 43, 11, 7, 37, 28, 31, 9, 20, 10, 41, 6, 14, 13, 22, 23, 15, 24, 19, 27, 44, 34, 3, 42, 2, 35, 29, 30, 40, 36, 21, 26, 4, 45, 5, 32, 1, 16, 33, 47, 25, 8, 12]
Długość trasy: 47942

### Długość listy tabu

In [173]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 5, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [17, 24, 47, 42, 7, 13, 41, 12, 14, 22, 21, 18, 37, 20, 26, 39, 32, 46, 30, 36, 45, 33, 28, 8, 44, 3, 25, 2, 29, 23, 11, 40, 16, 9, 34, 0, 31, 1, 27, 19, 6, 35, 38, 4, 5, 15, 43, 10]
Długość trasy: 47469


Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [17, 24, 47, 42, 7, 13, 41, 12, 14, 22, 21, 18, 37, 20, 26, 39, 32, 46, 30, 36, 45, 33, 28, 8, 44, 3, 25, 2, 29, 23, 11, 40, 16, 9, 34, 0, 31, 1, 27, 19, 6, 35, 38, 4, 5, 15, 43, 10]
Długość trasy: 47469

In [189]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 8, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 8, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [2, 35, 11, 45, 23, 24, 14, 46, 32, 33, 18, 28, 0, 42, 6, 7, 21, 43, 17, 31, 29, 26, 20, 38, 13, 8, 27, 40, 1, 41, 9, 36, 39, 4, 16, 44, 34, 37, 19, 22, 3, 25, 30, 5, 10, 15, 12, 47]
Długość trasy: 44110


In [175]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 15, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 15, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [32, 40, 33, 2, 11, 41, 6, 39, 14, 47, 7, 9, 24, 18, 1, 0, 23, 13, 8, 5, 37, 4, 3, 16, 22, 36, 38, 30, 31, 35, 17, 42, 20, 26, 21, 43, 29, 27, 44, 34, 25, 10, 19, 28, 15, 12, 45, 46]
Długość trasy: 49077


Rozmiar listy tabu: 15, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [32, 40, 33, 2, 11, 41, 6, 39, 14, 47, 7, 9, 24, 18, 1, 0, 23, 13, 8, 5, 37, 4, 3, 16, 22, 36, 38, 30, 31, 35, 17, 42, 20, 26, 21, 43, 29, 27, 44, 34, 25, 10, 19, 28, 15, 12, 45, 46]
Długość trasy: 49077

In [176]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 20, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [42, 7, 21, 17, 30, 27, 43, 1, 13, 10, 20, 31, 9, 23, 18, 11, 25, 3, 28, 44, 4, 38, 19, 36, 16, 34, 47, 29, 32, 37, 5, 15, 41, 45, 2, 33, 12, 6, 8, 14, 24, 39, 46, 0, 35, 22, 26, 40]
Długość trasy: 41160


Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [42, 7, 21, 17, 30, 27, 43, 1, 13, 10, 20, 31, 9, 23, 18, 11, 25, 3, 28, 44, 4, 38, 19, 36, 16, 34, 47, 29, 32, 37, 5, 15, 41, 45, 2, 33, 12, 6, 8, 14, 24, 39, 46, 0, 35, 22, 26, 40]
Długość trasy: 41160

### Liczba iteracji

In [177]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 100, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 100, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [14, 31, 27, 15, 0, 35, 4, 42, 29, 7, 24, 30, 3, 34, 1, 23, 21, 6, 22, 25, 44, 8, 16, 39, 41, 40, 33, 19, 17, 38, 43, 26, 12, 2, 10, 28, 9, 46, 47, 18, 13, 45, 11, 5, 36, 20, 37, 32]
Długość trasy: 45899


Rozmiar listy tabu: 10, maksymalna powtórzeń: 100, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [14, 31, 27, 15, 0, 35, 4, 42, 29, 7, 24, 30, 3, 34, 1, 23, 21, 6, 22, 25, 44, 8, 16, 39, 41, 40, 33, 19, 17, 38, 43, 26, 12, 2, 10, 28, 9, 46, 47, 18, 13, 45, 11, 5, 36, 20, 37, 32]
Długość trasy: 45899

In [178]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 1000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 1000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [14, 1, 29, 6, 0, 17, 20, 40, 4, 8, 37, 41, 2, 27, 47, 3, 5, 32, 46, 19, 18, 25, 13, 7, 42, 43, 24, 35, 10, 33, 11, 30, 36, 22, 28, 26, 38, 15, 9, 34, 45, 31, 12, 16, 44, 39, 23, 21]
Długość trasy: 53127


Rozmiar listy tabu: 10, maksymalna powtórzeń: 1000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [14, 1, 29, 6, 0, 17, 20, 40, 4, 8, 37, 41, 2, 27, 47, 3, 5, 32, 46, 19, 18, 25, 13, 7, 42, 43, 24, 35, 10, 33, 11, 30, 36, 22, 28, 26, 38, 15, 9, 34, 45, 31, 12, 16, 44, 39, 23, 21]
Długość trasy: 53127


In [179]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000,'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [21, 35, 31, 7, 27, 38, 37, 8, 0, 2, 11, 26, 42, 13, 46, 17, 30, 29, 3, 12, 44, 34, 1, 19, 23, 47, 14, 15, 39, 22, 25, 5, 18, 36, 43, 10, 16, 32, 4, 9, 33, 24, 20, 41, 45, 40, 6, 28]
Długość trasy: 43251


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [21, 35, 31, 7, 27, 38, 37, 8, 0, 2, 11, 26, 42, 13, 46, 17, 30, 29, 3, 12, 44, 34, 1, 19, 23, 47, 14, 15, 39, 22, 25, 5, 18, 36, 43, 10, 16, 32, 4, 9, 33, 24, 20, 41, 45, 40, 6, 28]
Długość trasy: 43251

In [180]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 100000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 100000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [45, 23, 32, 17, 24, 28, 10, 44, 33, 42, 7, 43, 6, 20, 14, 40, 30, 15, 41, 19, 21, 13, 47, 36, 5, 37, 31, 8, 2, 29, 25, 3, 22, 11, 26, 18, 34, 38, 16, 9, 12, 4, 0, 39, 46, 27, 1, 35]
Długość trasy: 50076


Rozmiar listy tabu: 10, maksymalna powtórzeń: 100000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [45, 23, 32, 17, 24, 28, 10, 44, 33, 42, 7, 43, 6, 20, 14, 40, 30, 15, 41, 19, 21, 13, 47, 36, 5, 37, 31, 8, 2, 29, 25, 3, 22, 11, 26, 18, 34, 38, 16, 9, 12, 4, 0, 39, 46, 27, 1, 35]
Długość trasy: 50076


### Czas liczenia

In [181]:
tabu_search_with_time_limit_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 10, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [47, 35, 0, 39, 1, 30, 13, 20, 10, 29, 15, 8, 40, 43, 4, 28, 21, 6, 24, 22, 9, 45, 2, 36, 14, 31, 26, 18, 44, 34, 25, 23, 17, 42, 37, 33, 19, 41, 16, 38, 32, 7, 5, 11, 27, 3, 12, 46]
Długość trasy: 48197


Rozmiar listy tabu: 10, maksymalny czas liczenia: 10, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [47, 35, 0, 39, 1, 30, 13, 20, 10, 29, 15, 8, 40, 43, 4, 28, 21, 6, 24, 22, 9, 45, 2, 36, 14, 31, 26, 18, 44, 34, 25, 23, 17, 42, 37, 33, 19, 41, 16, 38, 32, 7, 5, 11, 27, 3, 12, 46]
Długość trasy: 48197


In [182]:
tabu_search_with_time_limit_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 30, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [0, 15, 4, 18, 43, 34, 3, 22, 24, 38, 39, 19, 10, 45, 6, 32, 21, 1, 30, 29, 42, 41, 44, 7, 9, 47, 17, 13, 33, 16, 12, 14, 28, 26, 36, 5, 20, 25, 35, 8, 23, 2, 40, 11, 46, 37, 27, 31]
Długość trasy: 46809


Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [0, 15, 4, 18, 43, 34, 3, 22, 24, 38, 39, 19, 10, 45, 6, 32, 21, 1, 30, 29, 42, 41, 44, 7, 9, 47, 17, 13, 33, 16, 12, 14, 28, 26, 36, 5, 20, 25, 35, 8, 23, 2, 40, 11, 46, 37, 27, 31]
Długość trasy: 46809

In [183]:
tabu_search_with_time_limit_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 60, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [16, 26, 14, 19, 3, 36, 2, 11, 12, 6, 1, 17, 42, 38, 41, 47, 30, 24, 18, 39, 23, 40, 4, 0, 45, 15, 27, 32, 8, 9, 25, 29, 43, 20, 44, 34, 21, 28, 5, 33, 37, 22, 35, 10, 13, 46, 7, 31]
Długość trasy: 49835


Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [16, 26, 14, 19, 3, 36, 2, 11, 12, 6, 1, 17, 42, 38, 41, 47, 30, 24, 18, 39, 23, 40, 4, 0, 45, 15, 27, 32, 8, 9, 25, 29, 43, 20, 44, 34, 21, 28, 5, 33, 37, 22, 35, 10, 13, 46, 7, 31]
Długość trasy: 49835


In [184]:
tabu_search_with_time_limit_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 90, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 90, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [40, 36, 29, 16, 32, 31, 34, 19, 21, 1, 23, 46, 44, 33, 43, 35, 14, 7, 42, 28, 17, 4, 18, 12, 27, 25, 3, 24, 0, 5, 37, 13, 10, 6, 2, 39, 41, 38, 22, 26, 45, 8, 47, 30, 15, 9, 11, 20]
Długość trasy: 48849


Rozmiar listy tabu: 10, maksymalny czas liczenia: 90, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [40, 36, 29, 16, 32, 31, 34, 19, 21, 1, 23, 46, 44, 33, 43, 35, 14, 7, 42, 28, 17, 4, 18, 12, 27, 25, 3, 24, 0, 5, 37, 13, 10, 6, 2, 39, 41, 38, 22, 26, 45, 8, 47, 30, 15, 9, 11, 20]
Długość trasy: 48849


### Liczba wygenerowanych sąsiedztw

In [185]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000,'insert', 5, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 5
Najlepsze rozwiązanie spośród 5 prób: [19, 24, 13, 5, 26, 36, 21, 12, 4, 25, 3, 22, 10, 44, 28, 42, 46, 20, 33, 41, 29, 27, 14, 30, 17, 6, 15, 23, 16, 40, 47, 18, 2, 43, 11, 8, 37, 45, 0, 31, 7, 32, 1, 35, 34, 38, 39, 9]
Długość trasy: 45668


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 5
Najlepsze rozwiązanie spośród 5 prób: [19, 24, 13, 5, 26, 36, 21, 12, 4, 25, 3, 22, 10, 44, 28, 42, 46, 20, 33, 41, 29, 27, 14, 30, 17, 6, 15, 23, 16, 40, 47, 18, 2, 43, 11, 8, 37, 45, 0, 31, 7, 32, 1, 35, 34, 38, 39, 9]
Długość trasy: 45668


In [186]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000,'insert', 8, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 8
Najlepsze rozwiązanie spośród 5 prób: [35, 31, 46, 0, 40, 1, 3, 25, 29, 44, 10, 36, 27, 45, 14, 17, 22, 9, 32, 8, 28, 19, 37, 38, 7, 12, 20, 15, 5, 2, 11, 47, 23, 18, 39, 34, 41, 21, 16, 43, 13, 6, 33, 4, 30, 24, 42, 26]
Długość trasy: 47026


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 8
Najlepsze rozwiązanie spośród 5 prób: [35, 31, 46, 0, 40, 1, 3, 25, 29, 44, 10, 36, 27, 45, 14, 17, 22, 9, 32, 8, 28, 19, 37, 38, 7, 12, 20, 15, 5, 2, 11, 47, 23, 18, 39, 34, 41, 21, 16, 43, 13, 6, 33, 4, 30, 24, 42, 26]
Długość trasy: 47026


In [187]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000,'insert', 15, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 5 prób: [24, 45, 43, 22, 9, 37, 30, 29, 5, 16, 32, 15, 10, 3, 2, 7, 21, 25, 6, 27, 28, 36, 40, 4, 0, 20, 39, 35, 26, 11, 41, 19, 46, 17, 31, 1, 8, 18, 13, 33, 42, 44, 34, 38, 23, 12, 14, 47]
Długość trasy: 46261


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 5 prób: [24, 45, 43, 22, 9, 37, 30, 29, 5, 16, 32, 15, 10, 3, 2, 7, 21, 25, 6, 27, 28, 36, 40, 4, 0, 20, 39, 35, 26, 11, 41, 19, 46, 17, 31, 1, 8, 18, 13, 33, 42, 44, 34, 38, 23, 12, 14, 47]
Długość trasy: 46261


In [188]:
tabu_search_multiple_runs(TSP_48, create_initial_route(TSP_48), 10, 10000,'insert', 20, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [46, 37, 26, 2, 27, 20, 5, 33, 40, 31, 35, 18, 32, 4, 9, 34, 44, 14, 6, 1, 28, 21, 10, 12, 36, 39, 15, 19, 24, 3, 45, 43, 29, 8, 0, 11, 41, 23, 38, 25, 7, 42, 30, 16, 22, 13, 47, 17]
Długość trasy: 41851


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [46, 37, 26, 2, 27, 20, 5, 33, 40, 31, 35, 18, 32, 4, 9, 34, 44, 14, 6, 1, 28, 21, 10, 12, 36, 39, 15, 19, 24, 3, 45, 43, 29, 8, 0, 11, 41, 23, 38, 25, 7, 42, 30, 16, 22, 13, 47, 17]
Długość trasy: 41851


### Wnioski po testach: 
Testy parametrów dla zbioru 48 miast były prowadzone dla losowej trasy początkowej, wyniki pokazują, że algorytm tabu nie jest w stanie znaleźć transy nawet zbliżonej do optymalnej, gdy startowa droga jest stworzona losowo. 

Z funkcji sąsiedztw dla losowej/krótszej trasy najlepiej radzi sobie "insert", jeżeli chodzi o pozostałe parametry to na podstawie testu nie wygląda na to, że większa liczba iteracji / dłuższy czas liczenia prowadzi do lepszego rozwiązania. Zwiększenie długości liczby tabu nie daje jednolitych wniosków. Natomiast, zwiększenie generowanych sąsiedztw może prowadzić do poprawy wyniku. 

### Poszukiwanie najlepszego rozwiązania dla TSP_48
Do poszukania najlepszego rozwiązanie wykorzystamy trasę wyznaczoną przez algorytm NN oraz parametry, które najlepiej sprawdziły się w wyznaczaniu trasy bazującącej na losowym roziązaniu.

In [190]:
NN_route = [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 0, 7, 37, 30, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44] 
# Długość trasy:  12012

tabu_search_multiple_runs(TSP_48, NN_route, 20, 10000,'insert', 20, 5)

Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 0, 7, 37, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 11814


In [191]:
NN_improved_route = [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 0, 7, 37, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44] 
# Długość trasy: 11814
tabu_search_multiple_runs(TSP_48, NN_improved_route, 10, 10000,'swap', 10, 3)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 3 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 11545


In [192]:
NN_improved_route_2 = [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44] 
# Długość trasy: 11545
tabu_search_multiple_runs(TSP_48, NN_improved_route_2, 10, 10000,'swap', 10, 3)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 3 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 16, 42, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 11433


In [193]:
NN_improved_route_3 = [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 16, 42, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44] 
# Długość trasy: 11433
tabu_search_multiple_runs(TSP_48, NN_improved_route_3, 5, 10000,'swap', 15, 3)

Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 3 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 35, 27, 29, 5, 36, 18, 26, 16, 42, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 11397


In [13]:
NN_improved_route_4 = [9, 23, 41, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 30, 43, 17, 6, 35, 27, 29, 5, 36, 18, 26, 16, 42, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44] 
# Długość trasy: 11397
tabu_search_multiple_runs(TSP_48, NN_improved_route_4, 10, 10000,'reverse', 30, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 5 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 14, 32, 45, 30, 43, 17, 6, 35, 27, 29, 5, 36, 18, 26, 16, 42, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 11186


In [14]:
NN_improved_route_5 = [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 14, 32, 45, 30, 43, 17, 6, 35, 27, 29, 5, 36, 18, 26, 16, 42, 19, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
tabu_search_with_time_limit_multiple_runs(TSP_48, NN_improved_route_5, 10, 30,'reverse', 30, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 10986


In [15]:
NN_improved_route_6 = [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 21, 15, 2, 33, 40, 28, 1, 25, 3, 34, 44]
# Długość trasy: 10986
tabu_search_with_time_limit_multiple_runs(TSP_48, NN_improved_route_6, 10, 30,'reverse', 30, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 10982


In [16]:
NN_improved_route_7 = [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44]
# Długość trasy: 10982
tabu_search_with_time_limit_multiple_runs(TSP_48, NN_improved_route_7, 10, 30,'reverse', 30, 10)    # Brak poprawy

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 10982


In [17]:
tabu_search_with_time_limit_multiple_runs(TSP_48, NN_improved_route_7, 10, 30,'swap', 30, 10)    # Brak poprawy

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44]
Długość trasy: 10982


Wnioski: Najlepsza trasa jaką udało się znaleźć dla 48 miast ma długość 10982 i zawiera uszeregowanie miast w takiej kolejności [9, 23, 41, 4, 47, 38, 31, 20, 46, 12, 24, 13, 22, 10, 11, 19, 42, 16, 26, 18, 36, 5, 29, 27, 35, 6, 17, 43, 30, 45, 32, 14, 39, 8, 37, 7, 0, 15, 21, 2, 33, 40, 28, 1, 25, 3, 34, 44].
Do jej osiągnięcia potrzebne było wielokrotne uruchomienie algorytmu z coraz lepszą trasą, po szóstym razie występuje brak poprawy. 